## качаем данные

In [1]:
import requests
from bs4 import BeautifulSoup 
import time
from urllib.parse import urlparse, urljoin
from tqdm.notebook import tqdm

In [112]:
## найти id каждого вопроса на страницах
page = 1
main_url = 'https://zpp.rospotrebnadzor.ru/Forum/Appeals/AjaxindexList?page={page}&searchtext=&categories=[]'

def ids_from_page(soup):
    return [i.get('href') for i  in soup.findAll('a', {'class':'appeal-title-link'})]

number_pages = 500
urls = []
for page in tqdm(range(1, number_pages), desc='Pages'):
    time.sleep(0.5)
    res = requests.get(main_url.format(page=page))  
    soup = BeautifulSoup(res.text)
    urls.extend(ids_from_page(soup))
    
"""
<div class="col-md-12">
    <a href="/Forum/Appeals/Details/33375" class="appeal-title-link">
        <span class="custom-underline">
            Отказ в возврате денег за сертификат
        </span>
    </a>
</div>
"""

'\n<div class="col-md-12">\n    <a href="/Forum/Appeals/Details/33375" class="appeal-title-link">\n        <span class="custom-underline">\n            Отказ в возврате денег за сертификат\n        </span>\n    </a>\n</div>\n'

In [120]:
urls[:10]

['/Forum/Appeals/Details/33375',
 '/Forum/Appeals/Details/33365',
 '/Forum/Appeals/Details/33359',
 '/Forum/Appeals/Details/33358',
 '/Forum/Appeals/Details/33356',
 '/Forum/Appeals/Details/33355',
 '/Forum/Appeals/Details/33354',
 '/Forum/Appeals/Details/33352',
 '/Forum/Appeals/Details/33351',
 '/Forum/Appeals/Details/33348']

In [121]:
## имея id запросов достанем тексты на страницах
domain = 'https://zpp.rospotrebnadzor.ru'

def get_text_page_question(soup):
    return soup.findAll('p', {'class':'appeal-details-message'})[0].text
def get_text_page_answer(soup):
    return soup.findAll('p', {'class':'appeal-comments-message'})[0].text

question = []
answers = []

for url in tqdm(urls, desc='Pages'):
    time.sleep(0.5)
    res = requests.get(urljoin(domain, url))  
    soup = BeautifulSoup(res.text)
    
    question.append(get_text_page_question(soup))
    try:
        answers.append(get_text_page_answer(soup))
    except:
        answers.append('No answer')
  
"""
# https://zpp.rospotrebnadzor.ru/Forum/Appeals/Details/33375
...
<div class="row">
    <div class="col-md-12">
        <p class="appeal-details-message">
            24 мая 2022 года получил в подарок сертификат отдела мужской одежды магазина «ЦУМ»
            (ООО «Торговая компания «Сыктывкар», г. Сыктывкар, ул. Интернациональная, 147) номиналом в 2000 рублей.
            3 июня 2022 года обратился в магазин с целью возврата денежных средств за сертификат, на что получил отказ.
            При том что владелец сертификата может отказаться от него и потребовать у магазина возврата денежных средств
            согласно статье 421 Гражданского кодекса РФ о свободе договора.
            Так же в соответствии со статьей 1102 Гражданского кодекса РФ магазин обязан вернуть денежные средства,
            так как я не собираюсь воспользоваться сертификатом, а у магазина возникает неосновательное обогащение.
            Кроме того, на самом сертификате написано о невозможности его вернуть, что является нарушением закона,
            так как согласно определению Верховного суда от 27.12.2018 № 305-АД18-18522 ущемляет права потребителей.
            Ввиду вышеизложенного прошу обязать магазин вернуть мне денежные средства за сертификат
            и подвергнуть его взысканию за нарушение законодательства.
            А так же обязать исправить противоречащую закону
            и вводящую в заблуждение потребителя информацию на самом сертификате.
        </p>
    </div>
    <div class="col-md-12">
        <p class="appeal-element-bottom"><span> Опубликовано: </span> 03.06.2022 в 13:24</p>
    </div>
</div>
...
"""

In [122]:
## посмотрим, что скачалось
N = 2
print('Q:', question[N])
print()
print('A:', answers[N])

Q: Добрый день.
Купила в магазине техники электросамокат,попробовать как он ездит,в магазине не разрешили,попросила продавца рассказать про товар,она сказала,что она о нем ничего не знает,колеса..руль...газ..тормоз,сигнал горит,в течение двух недель можете вернуть - это все что было сказано.Попробовала дома его,а у этой модели практически нет амортизации и жесткий ход колеса и сложить самой его очень трудно.Приехала в магазин через 4 дня,где мне отказали его взять,сославшись на то,что этот товар с электродвигателем и входит в перечень товаров,которые нельзя вернуть,а только обменять на аналогичный,но в магазине не было такой модели с похожими характеристиками.Мне посоветовали написать претензию и вернули товар обратно.Правомерны ли действия магазина?и если не правомерны,то можно ссылку на закон?

A: Добрый день! Повторно, согласно ст.25 Закона о защите прав потребителей,от 07.02.1992 № 2300-1,  Потребитель вправе обменять непродовольственный товар надлежащего качества на аналогичный то

In [123]:
print(type(question))
print(len(question))
print(len(answers))

<class 'list'>
2495
2495


In [124]:
import pandas as pd

In [125]:
columns = ['Text'] 
question_df = pd.DataFrame(question)
question_df.head()

,0
0,24 мая 2022 года получил в подарок сертификат ...
1,Добрый день! помогите пожалуйста разобраться в...
2,Добрый день.\r\nКупила в магазине техники элек...
3,Добрый день.\r\nКупила в магазине техники элек...
4,Добрый день!\r\nПрошу вас разъяснить следующий...


In [126]:
answers_df = pd.DataFrame(answers)
answers_df.head()

,0
0,No answer
1,Уважаемая Арина!\r\nЕсли Вы заключили договор ...
2,"Добрый день! Повторно, согласно ст.25 Закона о..."
3,Добрый день! Прежде всего технически сложные т...
4,Управление Роспотребнадзора по Ярославской обл...


In [127]:
question_df.to_csv("./data/rpn_question.csv", header=False, index=False, encoding='utf-8')  # Сохранение без индексации
answers_df.to_csv("./data/rpn_answers.csv", header=False, index=False, encoding='utf-8')  # Сохранение без индексации

In [128]:
combine_df = pd.read_csv("./data/rpn_question.csv", names = columns)
combine_df.head()

,Text
0,24 мая 2022 года получил в подарок сертификат ...
1,Добрый день! помогите пожалуйста разобраться в...
2,Добрый день.\r\nКупила в магазине техники элек...
3,Добрый день.\r\nКупила в магазине техники элек...
4,Добрый день!\r\nПрошу вас разъяснить следующий...


In [129]:
combine_df = pd.read_csv("./data/rpn_answers.csv", names = columns)
combine_df.head()

,Text
0,No answer
1,Уважаемая Арина!\r\nЕсли Вы заключили договор ...
2,"Добрый день! Повторно, согласно ст.25 Закона о..."
3,Добрый день! Прежде всего технически сложные т...
4,Управление Роспотребнадзора по Ярославской обл...


In [55]:
# import csv

# with open("./data/rpn_question.csv", "w", newline="", encoding='utf-8') as f:
#     writer = csv.writer(f)
#     writer.writerows(question)

In [56]:
# with open("./data/rpn_answers.csv", "w", newline="", encoding='utf-8') as f:
#     writer = csv.writer(f)
#     writer.writerows(answers)